In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida
from models.polishing import polishing
from models.filtering import filtering

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_covid()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.1)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (361, 80)
19.75% missing data
Class distribution: (array([0, 1]), array([195, 166]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': [80, 80],
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 4001, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 0.54 sec - Loss 0.102094 - ACC 78.43% - ACC Mean 78.43% - AUC 87.48% - AUC Mean 87.48% - Deter 000
Ite 00050 - 0.10 sec - Loss 0.095703 - ACC 79.41% - ACC Mean 79.06% - AUC 87.84% - AUC Mean 87.39% - Deter 012
Ite 00100 - 0.11 sec - Loss 0.079324 - ACC 81.60% - ACC Mean 80.83% - AUC 88.97% - AUC Mean 88.51% - Deter 014
Ite 00150 - 0.10 sec - Loss 0.053546 - ACC 80.44% - ACC Mean 80.51% - AUC 88.18% - AUC Mean 88.58% - Deter 064
Ite 00200 - 0.11 sec - Loss 0.039317 - ACC 79.57% - ACC Mean 80.51% - AUC 88.39% - AUC Mean 88.53% - Deter 012
Ite 00250 - 0.49 sec - Loss 0.035540 - ACC 80.48% - ACC Mean 80.38% - AUC 88.28% - AUC Mean 88.30% - Deter 062
Ite 00300 - 0.10 sec - Loss 0.034372 - ACC 80.48% - ACC Mean 80.51% - AUC 88.28% - AUC Mean 88.26% - Deter 112
Ite 00350 - 0.14 sec - Loss 0.033789 - ACC 80.27% - ACC Mean 80.63% - AUC 88.18% - AUC Mean 88.19% - Deter 162
Ite 00400 - 0.17 sec - Loss 0.033346 - ACC 80.83% - ACC Mean 80.57% - AUC 88.11% - AUC Mean 87.86% - Deter 212
I

## Run MICE

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 79.73% +- 0.0% - auc: 88.37% +- 0.0%


## Run SFIL

In [6]:
sfil_accs, sfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='standard', random_state=i)
    sfil_accs.append(acc.mean())
    sfil_aucs.append(auc.mean())
    print(f'SFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SFIL GLOBAL - acc: {round(np.mean(sfil_accs) * 100, 2)}% +- {round(np.std(sfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(sfil_aucs) * 100, 2)}% +- {round(np.std(sfil_aucs) * 100, 2)}%')

SFIL RUN 1/10 - acc: 79.86% +- 2.0% - auc: 84.78% +- 4.6%
SFIL RUN 2/10 - acc: 79.71% +- 4.4% - auc: 84.11% +- 4.8%
SFIL RUN 3/10 - acc: 78.43% +- 2.86% - auc: 83.89% +- 3.55%
SFIL RUN 4/10 - acc: 79.88% +- 7.24% - auc: 86.71% +- 5.35%
SFIL RUN 5/10 - acc: 79.42% +- 3.24% - auc: 84.11% +- 3.28%
SFIL RUN 6/10 - acc: 79.12% +- 4.86% - auc: 81.56% +- 3.14%
SFIL RUN 7/10 - acc: 79.28% +- 4.28% - auc: 83.52% +- 1.9%
SFIL RUN 8/10 - acc: 81.02% +- 3.25% - auc: 85.76% +- 4.75%
SFIL RUN 9/10 - acc: 76.11% +- 6.64% - auc: 83.15% +- 4.06%
SFIL RUN 10/10 - acc: 78.39% +- 3.72% - auc: 82.48% +- 4.23%
SFIL GLOBAL - acc: 79.12% +- 1.24% - auc: 84.01% +- 1.42%


## Run PFIL

In [7]:
pfil_accs, pfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='panda', random_state=i)
    pfil_accs.append(acc.mean())
    pfil_aucs.append(auc.mean())
    print(f'PFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PFIL GLOBAL - acc: {round(np.mean(pfil_accs) * 100, 2)}% +- {round(np.std(pfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(pfil_aucs) * 100, 2)}% +- {round(np.std(pfil_aucs) * 100, 2)}%')

PFIL RUN 1/10 - acc: 79.88% +- 5.64% - auc: 84.55% +- 5.71%
PFIL RUN 2/10 - acc: 81.55% +- 3.3% - auc: 88.15% +- 2.8%
PFIL RUN 3/10 - acc: 83.2% +- 4.15% - auc: 88.76% +- 2.59%
PFIL RUN 4/10 - acc: 82.74% +- 2.95% - auc: 89.54% +- 1.83%
PFIL RUN 5/10 - acc: 82.93% +- 4.69% - auc: 88.75% +- 5.06%
PFIL RUN 6/10 - acc: 81.89% +- 3.61% - auc: 88.35% +- 2.82%
PFIL RUN 7/10 - acc: 81.93% +- 2.51% - auc: 87.54% +- 4.78%
PFIL RUN 8/10 - acc: 81.21% +- 4.8% - auc: 88.76% +- 3.74%
PFIL RUN 9/10 - acc: 80.91% +- 3.08% - auc: 87.3% +- 3.86%
PFIL RUN 10/10 - acc: 84.11% +- 4.08% - auc: 89.74% +- 2.61%
PFIL GLOBAL - acc: 82.04% +- 1.18% - auc: 88.14% +- 1.4%


## Run SPOL

In [8]:
spol_accs, spol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='standard', random_state=i)
    spol_accs.append(acc.mean())
    spol_aucs.append(auc.mean())
    print(f'SPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SPOL GLOBAL - acc: {round(np.mean(spol_accs) * 100, 2)}% +- {round(np.std(spol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(spol_aucs) * 100, 2)}% +- {round(np.std(spol_aucs) * 100, 2)}%')

SPOL RUN 1/10 - acc: 80.14% +- 4.66% - auc: 85.83% +- 4.54%
SPOL RUN 2/10 - acc: 80.65% +- 4.63% - auc: 85.49% +- 5.1%
SPOL RUN 3/10 - acc: 78.43% +- 3.57% - auc: 84.38% +- 2.02%
SPOL RUN 4/10 - acc: 83.5% +- 1.78% - auc: 87.73% +- 0.74%
SPOL RUN 5/10 - acc: 81.55% +- 3.84% - auc: 87.73% +- 3.37%
SPOL RUN 6/10 - acc: 81.2% +- 6.36% - auc: 87.66% +- 3.79%
SPOL RUN 7/10 - acc: 79.96% +- 3.84% - auc: 85.37% +- 3.44%
SPOL RUN 8/10 - acc: 78.55% +- 4.76% - auc: 86.21% +- 4.58%
SPOL RUN 9/10 - acc: 80.73% +- 2.56% - auc: 83.36% +- 4.21%
SPOL RUN 10/10 - acc: 79.41% +- 3.29% - auc: 88.4% +- 3.39%
SPOL GLOBAL - acc: 80.41% +- 1.43% - auc: 86.21% +- 1.56%


## Run PPOL

In [9]:
ppol_accs, ppol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='panda', random_state=i)
    ppol_accs.append(acc.mean())
    ppol_aucs.append(auc.mean())
    print(f'PPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PPOL GLOBAL - acc: {round(np.mean(ppol_accs) * 100, 2)}% +- {round(np.std(ppol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(ppol_aucs) * 100, 2)}% +- {round(np.std(ppol_aucs) * 100, 2)}%')

PPOL RUN 1/10 - acc: 82.25% +- 5.35% - auc: 88.29% +- 4.69%
PPOL RUN 2/10 - acc: 79.19% +- 4.66% - auc: 85.35% +- 6.11%
PPOL RUN 3/10 - acc: 83.05% +- 1.82% - auc: 88.46% +- 1.3%
PPOL RUN 4/10 - acc: 80.26% +- 1.07% - auc: 87.15% +- 3.01%
PPOL RUN 5/10 - acc: 82.4% +- 2.39% - auc: 88.55% +- 4.21%
PPOL RUN 6/10 - acc: 81.02% +- 3.36% - auc: 88.6% +- 1.91%
PPOL RUN 7/10 - acc: 81.62% +- 4.03% - auc: 86.54% +- 4.56%
PPOL RUN 8/10 - acc: 76.56% +- 4.79% - auc: 84.71% +- 6.53%
PPOL RUN 9/10 - acc: 80.14% +- 3.64% - auc: 88.38% +- 3.53%
PPOL RUN 10/10 - acc: 79.48% +- 1.37% - auc: 85.21% +- 1.77%
PPOL GLOBAL - acc: 80.6% +- 1.82% - auc: 87.13% +- 1.48%


## Run T-tests

In [10]:
for model, metrics in {
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'SFIL': {'ACC': sfil_accs, 'AUC': sfil_aucs},
        'PFIL': {'ACC': pfil_accs, 'AUC': pfil_aucs},
        'SPOL': {'ACC': spol_accs, 'AUC': spol_aucs},
        'PPOL': {'ACC': ppol_accs, 'AUC': ppol_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than SFIL
Metric AUC - OUR METHOD is significantly better than SFIL
Metric ACC - OUR METHOD is even with PFIL
Metric AUC - OUR METHOD is significantly better than PFIL
Metric ACC - OUR METHOD is significantly better than SPOL
Metric AUC - OUR METHOD is significantly better than SPOL
Metric ACC - OUR METHOD is significantly better than PPOL
Metric AUC - OUR METHOD is significantly better than PPOL
